# Groq API Tasks: Conversation Management & Information Extraction
**Submitted by: Daksh Agarwal**
***
This notebook demonstrates two core tasks using the Groq API with its OpenAI-compatible SDK, without relying on external frameworks like LangChain.

* **Task 1:** Managing and periodically summarizing a conversation history to maintain context over long interactions.
* **Task 2:** Extracting structured information (e.g., name, email, location) from unstructured chat messages using a JSON schema and the tool-calling feature.

### Instructions for Evaluation
To run this notebook, you must configure your own Groq API key.

1.  In the Colab menu, click the **key icon (Secrets)** on the left sidebar.
2.  Create a new secret with the name `GROQ_API_KEY`.
3.  Paste your personal Groq API key into the **Value** field.
4.  Ensure the **"Notebook access"** toggle is enabled.

In [ ]:
# %%capture
# @title 1. Setup and Dependencies
# Install the necessary Python library for Groq
!pip install groq

import os
from groq import Groq
from google.colab import userdata
import json

print("✅ Setup Complete: Libraries installed and imported.")

# --- API Key Configuration ---
# Securely fetch the API key from Colab's secret manager
try:
    groq_api_key = userdata.get('GROQ_API_KEY')
    os.environ["GROQ_API_KEY"] = groq_api_key
    print("✅ Groq API Key loaded successfully.")
except Exception as e:
    print("🚨 Error loading Groq API Key.")
    print("Please ensure you have set the 'GROQ_API_KEY' in Colab's Secrets Manager.")
    # Fallback for demonstration if key isn't set, but API calls will fail.
    groq_api_key = "YOUR_GROQ_API_KEY"


# --- Initialize Groq Client ---
try:
    client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
    print("✅ Groq client initialized.")
except Exception as e:
    client = None
    print(f"🚨 Failed to initialize Groq client: {e}")

# %%
# @title 2. Helper Functions for History Management

def format_history_for_display(history):
    """Helper to pretty-print the conversation history."""
    formatted_string = "--- Conversation History ---\n"
    if not history:
        return formatted_string + "  [Empty]\n"
    for msg in history:
        formatted_string += f"  [{msg['role'].capitalize()}]: {msg['content']}\n"
    formatted_string += "--------------------------"
    return formatted_string

def get_assistant_response(messages):
    """
    Gets a response from the Groq API based on the current conversation.
    This simulates the assistant's turn in a real chat application.
    """
    if not client:
        return "Groq client not initialized. Cannot get response."
    try:
        chat_completion = client.chat.completions.create(
            messages=messages,
            # ✅ CORRECTED MODEL NAME
            model="llama-3.1-8b-instant",
            temperature=0.7,
            max_tokens=100,
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        return f"Error communicating with Groq API: {e}"


# --- Truncation Functions ---

def truncate_by_turns(history, n_turns):
    """
    Limits the history to the last n conversation turns.
    A "turn" consists of one user message and one assistant message.
    """
    # Each turn has 2 messages (user + assistant)
    max_messages = n_turns * 2
    return history[-max_messages:]

def truncate_by_length(history, max_chars):
    """
    Limits the history by a maximum character count, starting from the most recent message.
    """
    truncated_history = []
    current_chars = 0
    # Iterate backwards from the most recent message
    for message in reversed(history):
        message_chars = len(message.get('content', ''))
        if current_chars + message_chars <= max_chars:
            # Prepend the message to maintain chronological order
            truncated_history.insert(0, message)
            current_chars += message_chars
        else:
            # Stop when the next message would exceed the limit
            break
    return truncated_history


# --- Summarization Function ---

def summarize_conversation(history):
    """
    Uses the Groq API to summarize the conversation history.
    """
    if not client:
        return "Groq client not initialized. Cannot summarize."

    # Convert the history to a simple string format for the prompt
    conversation_text = "\n".join([f"{msg['role']}: {msg['content']}" for msg in history])

    # Create a new set of messages for the summarization request
    summarization_prompt = [
        {
            "role": "system",
            "content": "You are an expert at summarizing conversations. Your task is to create a concise, neutral summary of the following chat history. Capture the main topics, key questions, and any conclusions reached. The summary should be a single, dense paragraph."
        },
        {
            "role": "user",
            "content": conversation_text
        }
    ]

    try:
        chat_completion = client.chat.completions.create(
            messages=summarization_prompt,
            # ✅ CORRECTED MODEL NAME
            model="llama-3.1-8b-instant",
        )
        summary = chat_completion.choices[0].message.content
        return summary
    except Exception as e:
        return f"Error during summarization: {e}"


# %%
# @title 3. Demonstration of Truncation Functionality

# --- Sample Conversation History ---
sample_history = [
    {'role': 'user', 'content': 'Hi, I want to plan a trip to Japan.'},
    {'role': 'assistant', 'content': 'Of course! When are you thinking of going and what are your interests?'},
    {'role': 'user', 'content': 'I was thinking next April to see the cherry blossoms. I love food and history.'},
    {'role': 'assistant', 'content': 'April is a perfect time for cherry blossoms! For food and history, I recommend a route covering Tokyo, Kyoto, and Osaka.'},
    {'role': 'user', 'content': 'Great! What are the must-see historical sites in Kyoto?'},
    {'role': 'assistant', 'content': 'In Kyoto, you shouldn\'t miss Kinkaku-ji (the Golden Pavilion), Fushimi Inari Shrine, and the Arashiyama Bamboo Grove.'},
    {'role': 'user', 'content': 'Excellent, thank you for the suggestions.'},
    {'role': 'assistant', 'content': 'You\'re welcome! Let me know if you need help with hotel or flight bookings.'}
]

print(" demonstrating truncation functionality ".center(60, "="))

# --- a. Limit by Number of Conversation Turns ---
print("\n### Truncation by Number of Turns (last 2 turns) ###\n")
truncated_by_turn = truncate_by_turns(sample_history, n_turns=2)
print("Original History Length (messages):", len(sample_history))
print("Truncated History Length (messages):", len(truncated_by_turn))
print(format_history_for_display(truncated_by_turn))

# --- b. Limit by Character/Word Length ---
print("\n### Truncation by Character Length (max 300 chars) ###\n")
truncated_by_len = truncate_by_length(sample_history, max_chars=300)
total_chars = sum(len(m['content']) for m in truncated_by_len)
print("Original History Length (messages):", len(sample_history))
print("Truncated History Length (messages):", len(truncated_by_len))
print(f"Total Characters in Truncated History: {total_chars}")
print(format_history_for_display(truncated_by_len))

print("\n" + "="*60)

# %%
# @title 4. Conversation Manager with Periodic Summarization

class ConversationManager:
    """
    Manages the conversation history, including periodic summarization.
    """
    def __init__(self, k_runs_for_summarization=3):
        """
        Initializes the manager.
        Args:
            k_runs_for_summarization (int): Summarize after every k-th run.
        """
        self.history = []
        self.run_counter = 0
        self.k = k_runs_for_summarization
        print(f"ConversationManager initialized. Summarization will occur after every {self.k} turns.")

    def add_turn(self, user_input):
        """
        Handles a single turn of the conversation.
        1. Adds user message.
        2. Gets and adds assistant message.
        3. Increments counter.
        4. Checks if it's time to summarize.
        """
        print(f"\n--- Turn {self.run_counter + 1} ---")

        # 1. Add user message
        self.history.append({"role": "user", "content": user_input})

        # 2. Get and add assistant's response
        # We pass the current history so the assistant has context
        assistant_response = get_assistant_response(self.history)
        self.history.append({"role": "assistant", "content": assistant_response})

        print(f"[User]: {user_input}")
        print(f"[Assistant]: {assistant_response}")

        # 3. Increment the run counter
        self.run_counter += 1

        # 4. Check for periodic summarization
        if self.run_counter % self.k == 0:
            print(f"\n>>> Condition met: Run {self.run_counter} is a multiple of {self.k}. Triggering summarization...")
            self.summarize_and_replace()

    def summarize_and_replace(self):
        """
        Summarizes the current history and replaces it with the summary.
        """
        print(">>> Current history before summarization:")
        print(format_history_for_display(self.history))

        # Get the summary
        summary = summarize_conversation(self.history)

        print("\n>>> Generated Summary:")
        print(summary)

        # Replace the entire history with a single system message containing the summary
        self.history = [
            {"role": "system", "content": f"This is a summary of the previous conversation: {summary}"}
        ]

        print("\n>>> History has been replaced with the summary.")
        print(format_history_for_display(self.history))

# %%
# @title 5. Demonstration of Periodic Summarization in Action (k=3)

print(" demonstrating periodic summarization ".center(60, "="))

# --- Sample Conversation Flow ---
conversation_flow = [
    "I need to learn about large language models. Where should I start?",
    "What's the difference between fine-tuning and retrieval-augmented generation?",
    "That makes sense. So RAG is better for knowledge-intensive tasks without retraining?",
    "Okay, let's switch topics. I want to build a simple Python web app.",
    "Which framework is easier for a beginner, Flask or Django?",
    "Thanks. I'll start with Flask. Can you give me a 'Hello, World!' example?",
    "Perfect! Final question for now: how do I deploy it easily?",
]

# Initialize the manager to summarize after every 3rd run
manager = ConversationManager(k_runs_for_summarization=3)

# Run the conversation flow
for user_prompt in conversation_flow:
    # Add a guard to stop if the client wasn't initialized
    if not client:
        print("\n🚨 Halting demonstration because Groq client is not initialized.")
        break
    manager.add_turn(user_prompt)
    print("\nCurrent History State:")
    print(format_history_for_display(manager.history))

print("\n" + "="*60)
print("✅ Demonstration Finished.")

✅ Setup Complete: Libraries installed and imported.
✅ Groq API Key loaded successfully.
✅ Groq client initialized.
========== demonstrating truncation functionality ==========

### Truncation by Number of Turns (last 2 turns) ###

Original History Length (messages): 8
Truncated History Length (messages): 4
--- Conversation History ---
  [User]: Great! What are the must-see historical sites in Kyoto?
  [Assistant]: In Kyoto, you shouldn't miss Kinkaku-ji (the Golden Pavilion), Fushimi Inari Shrine, and the Arashiyama Bamboo Grove.
  [User]: Excellent, thank you for the suggestions.
  [Assistant]: You're welcome! Let me know if you need help with hotel or flight bookings.
--------------------------

### Truncation by Character Length (max 300 chars) ###

Original History Length (messages): 8
Truncated History Length (messages): 4
Total Characters in Truncated History: 288
--- Conversation History ---
  [User]: Great! What are the must-see historical sites in Kyoto?
  [Assistant]: In Kyot